In [1]:
import numpy as np
import glob
import pickle

In [2]:
import duplicate_audio.wavBasicRead as wbr
from duplicate_audio.fingerprinting import fingerprint, add_to_fingerprints
from duplicate_audio.searching import search_1channel_in_dictdb

In [ ]:
# If f_db exists, then load
# this will take time
f_db_dump_fp = "./data/fingerprints_db.dump"
f_db = pickle.load(open(f_db_dump_fp, 'rb'))

### Creating Fingerprint database

In [3]:
wav_paths = glob.glob("./resources/Drum-Breaks-Collection/*.wav")
len(wav_paths)

30

In [4]:
wav_dict_list = wbr.read_list_wav(wav_paths)

//anaconda/lib/python3.4/site-packages/scipy/io/wavfile.py:42: WavFileWarning: Unknown wave file format
  warnings.warn("Unknown wave file format", WavFileWarning)
//anaconda/lib/python3.4/site-packages/scipy/io/wavfile.py:179: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [5]:
# generate the fingerprints for song_id=0

# NOTE: do not run if db already exists

# f_db = fingerprint(wav_dict_list[0]["data"], wav_dict_list[0]["sampling_frequency"], 0)

In [6]:
for i in range(1, len(wav_dict_list)):
    f_db = add_to_fingerprints(wav_dict_list[i]["data"], 
                               wav_dict_list[i]["sampling_frequency"], 
                               f_db, i)

In [7]:
# NOTE:: CHANGE THE last_song_id if new songs are added to a loaded f_db
# This will take time

# f_db_dump_fp = "./data/fingerprints_db.dump"
# pickle.dump({"db": f_db, "last_song_id": len(wav_dict_list)}, open(f_db_dump_fp, 'wb'))

### Searching

In [8]:
len(f_db.keys())

289485

In [37]:
WAV_TEST_FILES = ["./resources/Drum-Breaks-Collection/30-drum-break-146-bpm.wav", 
                  "./resources/Drum-Breaks-Collection/3-drum-break-102-bpm.wav",
                  "./resources/VID_20151119_012841.wav",  # 30
                  "./resources/VID_20151119_014554 (online-audio-converter.com).wav",  # 19
                  "./resources/VID_20151119_015502 (online-audio-converter.com).wav"  #30
                  ]
WAV_FILE_PATH = WAV_TEST_FILES[-1]

wav_dict = wbr.read_single_wav(WAV_FILE_PATH)
wav_dict["data"].shape

(356564, 2)

In [21]:
# make a dummy query

t21 = 0.5  # seconds
t1 = 1.3
t2 = t1 + t21
channel = 0

query_fs = wav_dict["sampling_frequency"]
t1_, t2_ = round(t1*query_fs), round(t2*query_fs)
print(t2_, t1_)
query_data = wav_dict["data"][t1_:t2_+1, channel]



# query_data = query_data + np.random.normal(0, 1,
#                               query_data.size)

# query_data = query_data + \
#     np.random.normal(np.mean(wav_dict["data"][:, channel]),
#                      np.std(wav_dict["data"][:, channel]),
#                      query_data.size)


query_data.shape[0]/query_fs

79380 57330


0.5000226757369615

In [38]:
query_data = wav_dict["data"][:, 1]
query_fs = wav_dict["sampling_frequency"]

query_data.shape[0]/query_fs

8.085351473922902

In [39]:
votes, song, channel, t = \
    search_1channel_in_dictdb(query_data, query_fs, f_db)

print("Identified channel %d of song at path: \n%s\n"
      %(channel, wav_dict_list[song]["abs_file_path"]))
print("Best timestamp at %5.2f seconds, with %d votes"
      %(t, votes))

Identified channel 1 of song at path: 
/Users/myrmidon/Delve/studies-mi/mmar/duplicate_audio/resources/Drum-Breaks-Collection/30-drum-break-146-bpm.wav

Best timestamp at -3.58 seconds, with 1662 votes
